In [2]:

def add_your_project(project_url,beta=False):
    """
    add_your_project is a function that adds a project to the list of projects

    Parameters

    :param project_url: the url of the project
    :type project_url: str
    :return: None
    :rtype: None
    """
    project_name = project_url.split('/')[-1] # negative index to get last
    username = project_url.split('/')[-2] # get the username that came before the second to last /
    markdown_string = """<a href="https://github.com/{}/{}"><img width="278" src="https://denvercoder1-github-readme-stats.vercel.app/api/pin/?username={}&repo={}&theme=react&bg_color=1F222E&title_color=9ACD32&hide_border=true&icon_color=FF4500&show_icons=false" alt="{}"></a>""".format(username,project_name,username,project_name,project_name)
    if beta: #! Note: this is an experimental if section. Should default to the else until this is working.
        # summarize the project by getting the readme file and reading just the first paragraph of it and summarizing it.
        # step 1 get the text from the readme file using requests and beautiful soup (bs4) libraries and finding the first paragraph.
        text = get_text_from_url(project_url)
        # the first paragraph is the first line of the text
        first_paragraph = text.split('\n')[0] # split the text by new line and get the first element
        # if the first paragraph is less than 100 characters, then it's probably not a good summary, so move on to the next paragraph.
        if len(first_paragraph) < 100:
            first_paragraph = text.split('\n')[1]
        if len(first_paragraph) < 100:
            first_paragraph = text.split('\n')[2]
        if len(first_paragraph) < 100:
            first_paragraph = text.split('\n')[3]
        
        print(first_paragraph)
        # step 2 summarize the text using the summarize function from the gensim library
        summary = run_summarization(first_paragraph)
        # step 3 add the summary to the markdown string
        #markdown_string += """<p align="center">{}</p>""".format(summary)
    else:
        summary = input("Enter a short description of the project: ")
    table_row  = """| [{}](https://github.com/{}/{}) |  ![last commit](https://img.shields.io/github/last-commit/{}/{}) ![code size](https://img.shields.io/github/languages/code-size/{}/{}) ![commit activity](https://img.shields.io/github/commit-activity/m/{}/{}) ![issues](https://img.shields.io/github/issues/{}/{}) | {} |""".format(project_name,username,project_name,username,project_name,username,project_name,username,project_name,username,project_name,summary)
    
    # concatenate the two strings with four line breaks in between them
    result =  'Paste the block below into the the bottom row of the table in the project_board.md file.' + '\n\n' + markdown_string + '\n\n' + 'Paste the block below under the appropriate heading in the main readme.md file.' + '\n\n' + table_row
    return result


project_url = input("Enter the url of your project: ")
markdown_string = add_your_project(project_url)

print(markdown_string)

Paste the block below into the the bottom row of the table in the project_board.md file.

<a href="https://github.com/bitmonky/mkyNetwork"><img width="278" src="https://denvercoder1-github-readme-stats.vercel.app/api/pin/?username=bitmonky&repo=mkyNetwork&theme=react&bg_color=1F222E&title_color=9ACD32&hide_border=true&icon_color=FF4500&show_icons=false" alt="mkyNetwork"></a>

Paste the block below under the appropriate heading in the main readme.md file.

| [mkyNetwork](https://github.com/bitmonky/mkyNetwork) |  ![last commit](https://img.shields.io/github/last-commit/bitmonky/mkyNetwork) ![code size](https://img.shields.io/github/languages/code-size/bitmonky/mkyNetwork) ![commit activity](https://img.shields.io/github/commit-activity/m/bitmonky/mkyNetwork) ![issues](https://img.shields.io/github/issues/bitmonky/mkyNetwork) | An experimental self organizing peer to peer network in node.js |


In [ ]:
# urls = ['https://github.com/talkygram/webcrawly','https://github.com/BKAmos/DataScience','https://github.com/VanillaLattA/DatascienceShortcuts']

# for url in urls:
#     markdown_string = create_project_tile(url)
#     print(markdown_string)